In [64]:
import tensorflow as tf
import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=18)
mpl.rc('xtick', labelsize=18)
mpl.rc('ytick', labelsize=18)
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')
import matplotlib.animation as animation

import collections
import statistics
import tensorflow as tf
from typing import Any, List, Sequence, Tuple

# from tf_env.UR_ENV import UR_env
from tf_env.UR_ENV import UR_env_simple

In [65]:
number_iterations = 100 # @param {type:"integer"}
max_anglular_velocity = 4
max_steps_env=40
discount = 0.99 # @param {type:"number"}


fc_layer_params=(300,300)
learning_rate = 1e-5 # @param {type:"number"}
number_eval_episodes = 3 # @param {type:"integer"}
collect_episodes_per_iteration = 5 # @param {type:"integer"}
# collect_max_steps = 20 # @param {type:"integer"}
replay_buffer_capacity = max_steps_env*collect_episodes_per_iteration # @param {type:"integer"}
log_interval = 1 # @param {type:"integer"}
eval_interval = 5 # @param {type:"integer"}

In [66]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
eps = np.finfo(np.float32).eps.item()# заменяет числа 0< минимальным неотрицательным

In [67]:
env= UR_env_simple(
    max_steps=max_steps_env,
    max_anglular_velocity=max_anglular_velocity)

e:\work\ENV\tf_env\UR_ENV.py:133: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  j_orient =orientation.as_euler('ZXZ',degrees=True)


In [68]:
class ActorCritic(tf.keras.Model):
    def __init__(
        self, 
        num_actions: int, 
        num_hidden_units: int):
        super().__init__()
        self.common = tf.keras.layers.Dense(num_hidden_units, activation = "relu")
        self.actor = tf.keras.layers.Dense(num_actions)
        self.critic = tf.keras.layers.Dense(1)
    
    def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
        x = self.common(inputs)
        return self.actor(x), self.critic(x)

In [69]:
num_actions = 6
num_hidden_units = 128

model = ActorCritic(num_actions, num_hidden_units)

In [70]:
def env_step(action, env):
    state, reward, done, discount = env.step(action)
    state = np.array(state,dtype=np.float32)
    reward = np.array(reward,dtype=np.float32)
    done = np.array(done,dtype=np.int32)# 0 first; 1 midle; 2 last
    discount = np.array(reward,dtype=np.float32)
    return (state,reward,done, discount)

def tf_env_step(action: tf.Tensor, env):
  return tf.numpy_function(env_step, [action, env], 
                           [tf.float32, tf.float32, tf.int32, tf.float32])

In [71]:
# model = ActorCritic(num_actions, num_hidden_units)

# initial_state = env.reset()
# initial_state = tf.constant(initial_state, dtype=tf.float32)


# state = initial_state
# print(state)
# state=tf.expand_dims(state,0)
# # state =tf.constant([[10.,20.,30.,10.,20.,30.]])

# for i in [1,2,3,4]:
#     action_logits_t, value = model(state)
#     print(action_logits_t)
#     print(value)
# action_logits_t, value = model(state)
# print(action_logits_t)
# print(value)

# action = np.clip(action_logits_t,-1,1)
# print(action)
# action_probs_t = tf.nn.softmax(action_logits_t)
# print(action_probs_t)

# action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
# action_probs = action_probs.write(0, action_probs_t[0])
# print(action_probs.read(0))

# values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
# values = values.write(0, tf.squeeze(value[0]))
# print(values.read(0))

# state, reward, done, discount = tf_env_step(action, env)


In [72]:
def run_episode(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int,
    env) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:

    action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    rewards = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)

    # state = tf.expand_dims(initial_state, 0)
    state = initial_state

    for t in tf.range(max_steps):
        # Convert state into a batched tensor (batch size = 1)        
        state = tf.expand_dims(state, 0)
        # Run the model and to get action probabilities and critic value
        action_logits_t, value = model(state)

        # Sample next action from the action probability distribution
        
        action = np.clip(action_logits_t,-1,1)      
        # action = tf.random.categorical(action_logits_t, 1)[0]
        action_probs_t = tf.nn.softmax(action_logits_t)

        # Store critic values
        values = values.write(t, tf.squeeze(value[0]))

        # Store log probability of the action chosen
        action_probs = action_probs.write(t, action_probs_t[0])

        # Apply action to the environment to get next state and reward
        state, reward, done, discount =env.step(action[0])
        # state.set_shape(initial_state_shape)

        # Store reward
        rewards = rewards.write(t, reward)

        if done==2:
            break

    action_probs = action_probs.stack()
    values = values.stack()
    rewards = rewards.stack()

    return action_probs, values, rewards

In [73]:
def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float, 
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Start from the end of `rewards` and accumulate reward sums
  # into the `returns` array
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward + gamma * discounted_sum
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) / 
               (tf.math.reduce_std(returns) + eps))

  return returns

In [74]:
huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

def compute_loss(
    action_probs: tf.Tensor,  
    values: tf.Tensor,  
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined Actor-Critic loss."""

  advantage = returns - values

  action_log_probs = tf.math.log(action_probs)

  actor_loss = -tf.math.reduce_sum(action_log_probs)


  critic_loss = huber_loss(values, returns)

  return actor_loss + critic_loss

In [75]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


# @tf.function
def train_step(
    initial_state: tf.Tensor, 
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer, 
    gamma: float, 
    max_steps_per_episode: int,
    env) -> tf.Tensor:
  """Runs a model training step."""

  with tf.GradientTape() as tape:

    # Run the model for one episode to collect training data
    action_probs, values, rewards = run_episode(
        initial_state, model, max_steps_per_episode, env) 

    # Calculate the expected returns
    # returns = get_expected_return(rewards, gamma)
    returns = rewards

    # Convert training data to appropriate TF tensor shapes
    # action_probs, values, returns = [
    #     tf.expand_dims(x, 1) for x in [action_probs, values, returns]] 

    # Calculate the loss values to update our network
    loss = compute_loss(action_probs, values, returns)

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  last_reward = rewards[-1]

  return last_reward

In [76]:
# max_episodes = 2
# max_steps_per_episode = 2



# # The discount factor for future rewards
# gamma = 0.99
# initial_state = env.reset()
# initial_state = tf.constant(initial_state, dtype=tf.float32)
# episode_reward = train_step(
#     initial_state, 
#     model, 
#     optimizer, 
#     gamma, 
#     max_steps_per_episode,
#     env)

In [77]:
# %%time
model = ActorCritic(num_actions, num_hidden_units)

max_episodes = 2
max_steps_per_episode = 2


# The discount factor for future rewards
gamma = 0.99

# Keep the last episodes reward
episodes_reward = []

for i in tf.range(max_episodes):
    initial_state = env.reset()
    initial_state = tf.constant(initial_state, dtype=tf.float32)
    episode_reward = train_step(
        initial_state, 
        model, 
        optimizer, 
        gamma, 
        max_steps_per_episode,
        env)

    episodes_reward.append(episode_reward)

    if i % 1 == 0:
      print(episode_reward, 'dfgdfdf')
      pass # print(f'Episode {i}: average reward: {avg_reward}')


# print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')

tf.Tensor(0.00914364, shape=(), dtype=float32) dfgdfdf


ValueError: cannot convert float NaN to integer